# Creation of in vitro insertion parts for pfa chain in C.Cinerea

Now that promoters have been collected and generated and ran through a quantative flouresence analysis we can create the full list of inserts and necesarry primers for the top 5 primers and terminators. 

This insert will contain: <br>
- PABA marker gene <br>
- PUFA synthesis pathway <br>
    - pfa1,2,3 and a pptase <br>
- Hygromicin marker gene. 

*this is where we insert a cool image of the creation*

## Loading Libraries

## Genetic elements

### HR Domain

The insertion site into C.Cinerea will be the spoII site based on the report: <br> *Non-conventional fungi are efficient heterologous hosts for natural product production.*

### PABA marker

### PUFA Synthesis pathway

### Hygromicin marker

## Primer creation using TEEMI

### Imports

In [1]:
from teemi.design.combinatorial_design import DesignAssembly

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from IPython.display import display
import IPython.core.display

import os
os.chdir("..")

### Fetching everything

In [ ]:
from src.smart_functions import read_fasta_to_dseqrecords

top_promoters = r'notebooks/data/insert_sequnces/promoters.fasta'
top_terminators = r'notebooks/data/insert_sequnces/terminators.fasta'

hr_fa= r'notebooks/data/insert_sequnces/HR.fasta'
m_paba_fa = r'notebooks/data/insert_sequnces/PABA.fasta'
m_hygro_fa = r'notebooks/data/insert_sequnces/Hygromycin.fasta' #hygromicin including promoter and terminator.
cds_fa = r'notebooks/data/insert_sequnces/pufa_optimized.fasta' #pfa123 and pptase

promoters, promoter_names = read_fasta_to_dseqrecords(top_promoters)
terminators, terminator_names = read_fasta_to_dseqrecords(top_terminators)

cds_records, cds_names = read_fasta_to_dseqrecords(cds_fa)
m_paba, m_paba_names = read_fasta_to_dseqrecords(m_paba_fa)
m_hygro, m_hygro_names = read_fasta_to_dseqrecords(m_hygro_fa)
hr_records, HR_names = read_fasta_to_dseqrecords(hr_fa)

#### A quick count of sequences

In [3]:
print(f"Promoters: {len(promoters)}, CDS: {len(cds_records)}, Terminators: {len(terminators)}")

Promoters: 9, CDS: 3, Terminators: 9


### Putting it all together in a list

In [ ]:
full_constructs = []

for p in promoters:
    for t in terminators:
        
        cds_block = []
        for cds in cds_records:
            cds_block.extend([p, cds, t])

        construct = [
            hr_records[0],
            m_paba[0],
            *cds_block,
            m_paba[1],
            m_hygro,
            hr_records[1]
        ]
        full_constructs.append(construct)

print(len(full_constructs), "constructs generated")

81 constructs generated


### Combining

In [ ]:
TARGET_TM = 55
LIMIT = 13
OVERLAP = 35


design = DesignAssembly(full_constructs, list_of_pads=[], positions_of_pads=[], target_tm=TARGET_TM, limit=LIMIT, overlap=OVERLAP)
variants_df = design.show_variants_lib_df()          
primers_df  = design.primer_list_to_dataframe()      
pcrs_df = design.pcr_list_to_dataframe()


out_dir = os.path.abspath(".")
variants_csv = os.path.join(out_dir, "variants_library.csv")
primers_csv  = os.path.join(out_dir, "primers_list.csv")
pcrs_csv= os.path.join(out_dir, "pcr_plan.csv")

variants_df.to_csv(variants_csv, index=False)
primers_df.to_csv(primers_csv, index=False)
pcrs_df.to_csv(pcrs_csv, index=False)

print(f"Variants: {len(variants_df)} saved -> {variants_csv}")
print(f"Primers:  {len(primers_df)} saved -> {primers_csv}")
print(f"PCRs:     {len(pcrs_df)} saved -> {pcrs_csv}")
display(variants_df.head())
display(primers_df.head())
display(pcrs_df.head())